## Gold analysis
Se il massimo di mercoledi, e' inferiore a quello di lunedi, vedrai il minimo di mercoledi rivisitato giovedi (rivisitato=giovedi il prezzo crossa dall'alto o dal basso il minimo di mercoledi)

### Step da fare:
- Calcolare dei massimi e minimi giornalieri
    - bisogna fare resampling dei dati (in modo da ottenere candele da 1 giorno)
- Categorizzare ogni giorno della settimana (bisogna creare una colonna che dice che giorno e')
- Controllare se il max di mercoledi e' inferiore a quello di lunedi'
    - salva il minimo di mercoledi'
    - controlla se il range di giovedi include il minimo di mercoledi'.

### Readint the CSV file and converting it to a parquet one:

In [ ]:
# #import the libraries
# import cudf
# import dask
# import dask.dataframe as dd
# #set the enviroment to cuDF so we use the GPU
# dask.config.set({"dataframe.backend": "cudf"})
# #----------------------------------------------

# xau1D = dd.read_csv('/home/edoardocame/Desktop/python_dir/xauusd-d1-bid-2014-01-01-2024-12-11T23.csv')
# xau1D['timestamp'] = dd.to_datetime(xau1D['timestamp'])
# xau1D = xau1D.set_index('timestamp', sorted=True)
# xau1D['weekday'] = xau1D.index.to_series().dt.weekday
# xau1D.head()

### Using parquet file:

In [1]:
#import the libraries
import cudf
import dask
import dask.dataframe as dd
#set the enviroment to cuDF so we use the GPU
dask.config.set({"dataframe.backend": "cudf"})
#----------------------------------------------

df = dd.read_parquet('/home/edoardocame/Desktop/python_dir/xauusd1D.parquet')
df['returns'] = df['close'].diff() / df['close'].shift(1)
df['week'] = df.index.dt.isocalendar().week
df.head()

,open,high,low,close,volume,weekday,returns,week
timestamp,,,,,,,,
2014-01-01,1203.612,1205.883,1202.302,1205.883,0.2705,2,<NA>,1
2014-01-02,1205.913,1230.773,1204.893,1223.710,27.3592,3,0.014783358,1
2014-01-03,1223.687,1240.153,1223.297,1236.683,26.3572,4,0.010601368,1
2014-01-05,1236.983,1238.353,1233.842,1234.042,0.3280,6,-0.002135551,1
2014-01-06,1234.042,1248.342,1214.626,1237.665,26.1419,0,0.002935881,2


Let's break down the code step-by-step:

1. **Create Sample DataFrame**:
   ```python
   import pandas as pd

   data = {
       'Date': pd.date_range(start='2023-01-01', periods=10, freq='D'),
       'High': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
       'DayOfWeek': ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday', 'Monday', 'Tuesday']
   }
   df = pd.DataFrame(data)
   ```

   This creates a DataFrame with columns `Date`, `High`, and `DayOfWeek`.

2. **Add a Week Number Column**:
   ```python
   df['Week'] = df['Date'].dt.isocalendar().week
   ```

   This adds a new column `Week` to the DataFrame, which contains the week number for each date.

3. **Pivot Table to Get Monday's and Wednesday's High Values**:
   ```python
   pivot_df = df.pivot_table(index='Week', columns='DayOfWeek', values='High', aggfunc='first')
   ```

   This creates a pivot table where:
   - `index='Week'` groups the data by week.
   - `columns='DayOfWeek'` creates columns for each day of the week.
   - `values='High'` fills the table with the `High` values.
   - `aggfunc='first'` ensures that if there are multiple entries for the same day, it takes the first one.

   The resulting `pivot_df` will look something like this:

   | Week | Monday | Wednesday | ... |
   |------|--------|-----------|-----|
   | 1    | 20     | 40        | ... |
   | 2    | 80     | NaN       | ... |

4. **Check if Wednesday's High is Lower than Monday's High**:
   ```python
   pivot_df['Wed_Lower_Than_Mon'] = pivot_df['Wednesday'] < pivot_df['Monday']
   ```

   This creates a new column `Wed_Lower_Than_Mon` in the pivot table, which contains `True` if Wednesday's high is lower than Monday's high for the same week, and `False` otherwise.

5. **Merge the Result Back to the Original DataFrame**:
   ```python
   df = df.merge(pivot_df['Wed_Lower_Than_Mon'], on='Week', how='left')
   ```

   This merges the `Wed_Lower_Than_Mon` column from the pivot table back into the original DataFrame based on the `Week` column.

The final DataFrame `df` will have a new column `Wed_Lower_Than_Mon` indicating whether Wednesday's high is lower than Monday's high for the same week.